In [1]:
import numpy as np
import pandas as pd

In [2]:
dataset = pd.read_csv("/Users/alejandropalacios/Desktop/Ironhack/Data Analytics Bootcamp/Advanced Data Analysis Techniques/Project/F1-Grand-Prix-Predictor/Data Manipulation/Modeling/Modeled Databases/final_df_complete.csv")

In [3]:
# The goal of this exercise will be, for every Driver, to modelate 0s for repeated Teams after every Season and 1s for different Teams. Thus we will get a binary idea of how many times a Driver switched Teams between Seasons.

# Furthermore, we will do the exact same but for Teams, we will see how many times a Team switched Drivers between seasons. 

# Step one, pulverize the problem into various subsets of things. Let's only get the Drivers, their Season, and with which Constructor they drove. 

drivers = dataset.groupby('driver').apply(lambda a: a.drop('driver', axis=1)[:])

df = drivers.reset_index().drop(['level_1'], axis=1)

team_change_dataset = df.sort_values(by=['season','constructor','driver']).reset_index().drop(['index'], axis=1)

# Let's check Hamilton's career. Please note the fact that having duplicates does not mean we are going through a bad path, on the contrary, we need the duplicates to codify every row to a 0 or 1 value later.

team_change_dataset[team_change_dataset['driver'] == 'hamilton']


,driver,season,round,circuit_id,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,nationality,...,driver_standings_pos_after_race.1,driver_points,driver_wins,driver_standings_pos,constructor_points,constructor_wins,constructor_standings_pos,qualifying_time,driver_age,wins_percentage
8753,hamilton,2007,1,albert_park,True,False,False,False,False,British,...,0,0,0,0,0,0,0,0.683,22,0.00
8754,hamilton,2007,2,sepang,False,False,False,False,False,British,...,3,0,0,0,14,0,11,1.002,22,0.00
8755,hamilton,2007,3,bahrain,False,False,False,False,False,British,...,3,6,0,3,32,1,11,0.283,22,0.00
8756,hamilton,2007,4,catalunya,True,False,False,False,False,British,...,3,14,0,3,44,1,11,0.364,22,0.00
8757,hamilton,2007,5,monaco,False,False,False,False,True,British,...,1,22,0,3,58,1,11,0.179,22,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13519,hamilton,2019,17,suzuka,True,False,False,False,False,British,...,1,296,8,1,571,11,1,0.238,34,0.00
13520,hamilton,2019,18,rodriguez,False,False,False,False,True,British,...,1,322,9,1,612,12,1,0.238,34,0.25
13521,hamilton,2019,19,americas,True,False,False,False,False,British,...,1,338,9,1,652,13,1,0.292,34,0.00
13522,hamilton,2019,20,interlagos,True,False,False,False,False,British,...,1,363,10,1,695,14,1,0.191,34,0.00


In [4]:
# Now here's the trick, we will use a where clause to codify our data.

team_change_dataset['change_teams'] = np.where((team_change_dataset['driver'] == team_change_dataset['driver'].shift(periods=-1,axis=0)) & (team_change_dataset['constructor'] == team_change_dataset['constructor'].shift(periods=-1,axis=0)),0,1)


In [5]:
# A little check

team_change_dataset[['driver','season','round','constructor','change_teams']][(team_change_dataset['driver']=='hamilton') & (team_change_dataset['constructor']=='mercedes')]

,driver,season,round,constructor,change_teams
11183,hamilton,2013,1,mercedes,0
11184,hamilton,2013,2,mercedes,0
11185,hamilton,2013,3,mercedes,0
11186,hamilton,2013,4,mercedes,0
11187,hamilton,2013,5,mercedes,0
...,...,...,...,...,...
13519,hamilton,2019,17,mercedes,0
13520,hamilton,2019,18,mercedes,0
13521,hamilton,2019,19,mercedes,0
13522,hamilton,2019,20,mercedes,0


In [6]:
# Now let's sort things out in order to be able to use the dataset as a time series.

team_change_dataset = team_change_dataset.sort_values(by=['season','round','constructor','driver']).reset_index().drop(['index'], axis=1)


In [7]:
team_change_dataset[['driver','season','round','constructor','change_teams']][(team_change_dataset['driver']=='albon') & (team_change_dataset['season']==2019)]

,driver,season,round,constructor,change_teams
13344,albon,2019,1,toro_rosso,0
13364,albon,2019,2,toro_rosso,0
13397,albon,2019,4,toro_rosso,0
13415,albon,2019,5,toro_rosso,0
13435,albon,2019,6,toro_rosso,0
13453,albon,2019,7,toro_rosso,0
13473,albon,2019,8,toro_rosso,0
13492,albon,2019,9,toro_rosso,0
13511,albon,2019,10,toro_rosso,0
13529,albon,2019,11,toro_rosso,0


In [8]:
team_change_dataset.to_csv(r'/Users/alejandropalacios/Desktop/Ironhack/Data Analytics Bootcamp/Advanced Data Analysis Techniques/Project/F1-Grand-Prix-Predictor/Data Manipulation/Modeling/Modeled Databases/team_change_dataset.csv', index = False)